prova
• 25% anàlisis dels atributs (data mining),
• 25% aplicació de diversos mètodes d'aprenentatge,
• 30% visualització i presentació dels resultats,
• 10% repositori github
• 10% presentació opcional